In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
import numpy as np 
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
from matplotlib import pyplot as plt

In [5]:
pd.set_option('display.max_columns', 400)
pd.set_option('display.max_rows', 30)

In [6]:
col_names = ['name', 'views_num', 'area', 'COORD_X', 'COORD_Y',
             'city', 'address', 'district', 'underground', 'rooms', 
             'floor', 'square', 'height', 'build_year', 'toilet',
             'balcony', 'pol', 'repair', 'price', 'terms_sale', 
             'property', 'shop', 'pharmacy', 'updated', 'agency', 'video']
col_names = [i.upper() for i in col_names]
col_names

df = pd.read_csv('realtby_end_3.0.csv')
df.drop(['Unnamed: 0'], axis=1, inplace=True)
# df.columns = col_names

In [7]:
df

,NAME,VIEWS_NUM,AREA,COORD_X,COORD_Y,CITY,ADDRESS,DISTRICT,UNDERGROUND,ROOMS,FLOOR,SQUARE,HEIGHT,BUILD_YEAR,TOILET,BALCONY,POL,REPAIR,PRICE,TERMS_SALE,PROPERTY,SHOP,PHARMACY,UPDATED,AGENCY,VIDEO,FULL
0,Продажа 1/2 доли в 3-комнатной квартире в г. Г...,"просмотров: сегодня – 3, за последние 7 дней ...",Гродненская область,30.247803,53.927870,г. Гродно,NaN,NaN,NaN,1/2 доли в 3-комнатной квартире,6 / 9,57 / 20 / 7 м²,2.52 м,1980.0,раздельный,балкон и лоджия,деревянные,без отделки,"21 526 руб, 1 076 руб/кв.м.",чистая продажа,гос.-приватизированная,~ 200,~ 500,2019-09-25,NaN,Фотографии и планировки объекта:;;,Ленинский район: Пересёлка-4
1,Продажа 1/2 доли в 3-комнатной квартире в г. Г...,"просмотров: сегодня – 4, за последние 7 дней ...",Гродненская область,23.816521,53.697233,г. Гродно,NaN,NaN,NaN,1/2 доли в 3-комнатной квартире,6 / 9,57 / 20 / 7 м²,2.52 м,1980.0,раздельный,балкон и лоджия,деревянные,без отделки,"21 526 руб, 1 076 руб/кв.м.",чистая продажа,гос.-приватизированная,~ 200,~ 500,2019-09-25,NaN,null;;,Ленинский район: Пересёлка-4
2,Продажа комнаты в квартире в Кобринском районе...,"просмотров: сегодня – 6, за последние 7 дней ...",Брестская область,30.247803,53.927870,г. Кобрин,NaN,NaN,NaN,47/100 доли в 2-комнатной квартире,5 / 5,49.1 / 14 / 7.2 м²,2.5 м,1979.0,раздельный,лоджия,NaN,плохое состояние,"22 551 руб, 1 611 руб/кв.м.",чистая продажа,частная,~ 200,~ 200,2019-08-23,АЛЬТЕРНАТИВА Брест,Фотографии и планировки объекта:;;,NaN
3,Продажа комнаты в квартире в Кобринском районе...,"просмотров: сегодня – 8, за последние 7 дней ...",Брестская область,24.353911,52.208929,г. Кобрин,NaN,NaN,NaN,47/100 доли в 2-комнатной квартире,5 / 5,49.1 / 14 / 7.2 м²,2.5 м,1979.0,раздельный,лоджия,NaN,плохое состояние,"22 551 руб, 1 611 руб/кв.м.",чистая продажа,частная,~ 200,~ 200,2019-08-23,АЛЬТЕРНАТИВА Брест,null;;,NaN
4,Продажа доли в 3-х комнатной квартире,"просмотров: сегодня – 12, за последние 7 дней...",Минская область,30.247803,53.927870,г. Марьина Горка,"Новая заря ул., 2",NaN,NaN,1/2 доли в 3-комнатной квартире,2 / 4,71.4 / 48.1 / 9.3 м²,NaN,1949.0,раздельный,лоджия,деревянные,нормальный ремонт,"22 551 руб, 469 руб/кв.м.",чистая продажа,NaN,~ 350,~ 700,2019-11-21,Квадратный метр,Фотографии и планировки объекта:;;,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12782,"Продается квартира 8 комнат д. Тарасово, ул. Р...","просмотров: сегодня – 3, за последние 7 дней ...",Минская область,27.391663,53.915300,д. Тарасово,"Ратомская ул., 1",NaN,NaN,8 / 8,2 этажа,254.7 / 151.8 / 61.3 м²,2.64 м,2012,2 сан.узла,лоджия,деревянные,отличный ремонт,"861 042 руб, 3 381 руб/кв.м.",чистая продажа,частная,NaN,NaN,2019-11-22,Агентство недвижимости Особый стиль,null;;;;,NaN
12783,Продается современная 2-х этажная квартира пен...,"просмотров: сегодня – 12, за последние 7 дней...",Минская область,27.677200,53.946800,г. Минск,"Городецкая ул., 32 Информация о доме",Первомайский район,"м. Уручье, расстояние ≈700м, (Московская линия)",8 / 7,7 / 9,304.3 / 211.3 / 19.8 м²,NaN,2000,3 сан.узла,3 балкона,паркет,евроремонт,"1,17 млн руб, 3 840 руб/кв.м.",чистая продажа,частная,~ 100,~ 200,2019-11-22,Тэтравита,null;;;;,Первомайский район: Уручье
12784,"Продажа 9-и комнатной квартиры, г. Минск, ул. ...","просмотров: сегодня – 15, за последние 7 дней...",Минская область,27.678500,53.883600,г. Минск,"Малое Стиклево ул., 5 Информация о доме",Заводской район,NaN,9 / 9,3 этажа,307.4 / 191.3 / – м²,3.2 м,2010,3 сан.узла,нет,ламинированные,нормальный ремонт,"389 519 руб, 1 267 руб/кв.м.",чистая продажа,частная,~ 300,~ 200,2019-09-26,Эксперт,null;;;;,Заводской район: Ангарская
12785,Продаётся уникальная 9к квартира в центре Минска.,"просмотров: сегодня – 13, за последние 7 дней...",Минская область,27.526400,53.913900,г. Минск,"Москвина ул., 8 Информация о доме",Центральный район,"м. Молодежная, расстояние ≈900м, (Заводская ли...",9 / 8,4 / 5,470 / 349 / 27.1 м²,2.98 м,2003,3 сан.узла,2 балкона,паркет,

In [8]:
# Works

class ViewsEncoder(BaseEstimator, TransformerMixin):
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        values = X['VIEWS_NUM']
        values = [re.findall(r'\d+', i)[0::2] if not pd.isna(i) else [] for i in values]
        for i in range(len(values)):
            for j in range(2 - len(values[i])):
                values[i].append(np.nan)
        nums = np.array([[float(j) for j in i] for i in values])
        X['VIEWS_TODAY'], X['VIEWS_LAST_WEEK'] = nums[:, 0], nums[:, 1]
        X.drop(['VIEWS_NUM'], axis=1, inplace=True)
        return X

In [9]:
ven = ViewsEncoder()
df = ven.fit_transform(df)

In [10]:
class NumericFilter(BaseEstimator, TransformerMixin):
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        for i in ['SHOP', 'PHARMACY', 'UNDERGROUND']:
            X[i] = X[i].map(self.preprocessing_handler_int)
        for i in ['HEIGHT']:
            X[i] = X[i].map(self.preprocessing_handler_float)
            
        
        
        values = X['FLOOR']
        values = [re.findall(r"[-+]?\d*\.\d+|\d+", i) if not pd.isna(i) else [] for i in values]
        for i in range(len(values)):
            for j in range(2 - len(values[i])):
                values[i].append(np.nan)
        values = [[float(j) for j in i] for i in values]
        values = np.array(values)
        X['FLOOR_NOW'] = values[:, 0]
        X['FLOOR_MAX'] = values[:, 1]
        X.drop(['FLOOR'], axis=1, inplace=True)
        

        values = X['SQUARE']
        values = [re.findall(r"[-+]?\d*\.\d+|\d+", i) if not pd.isna(i) else [] for i in values]
        for i in range(len(values)):
            for j in range(3 - len(values[i])):
                values[i].append(np.nan)
        values = [[float(j) for j in i] for i in values]
        values = np.array(values)
        X['SQUARE_SUM'] = values[:, 0]
        X['SQUARE_LIVE'] = values[:, 1]
        X['SQUAER_KITCHEN'] = values[:, 2]
        X.drop(['SQUARE'], axis=1, inplace=True)
        
        return X
    
    def preprocessing_handler_int(self, x):
        if pd.isna(x):
            return x
        items = re.findall(r'\d+', x)
        if len(items) < 1:
            return np.nan
        else:
            return int(items[0])
        
    def preprocessing_handler_float(self, x):
        if pd.isna(x):
            return x
        items = re.findall(r"[-+]?\d*\.\d+|\d+|\d*\,\d+", x)
        if len(items) < 1:
            return np.nan
        else:
            return float(items[0])

In [11]:
nf = NumericFilter()
df = nf.fit_transform(df)

In [12]:
# Works

class PriceTransformer(BaseEstimator, TransformerMixin):
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X['PRICE'] = X['PRICE'].map(self.price_prep)
        X = X[~X['PRICE'].isna()]
        return X
        
    def price_prep(self, s):   
        if pd.isna(s):
            return float('nan')
        if s == 'Цена договорная':
            return float('nan')

        if s.find('млн') != -1:        
            s = s[:s.find('млн')].replace('\xa0', '')        
            return int(float(s.replace(',', '.'))*1000000)

        return int(s.split(',')[0].replace('руб', '').replace('\xa0', ''))

    

In [13]:
ptr = PriceTransformer()
df = ptr.fit_transform(df)

In [14]:
# Works

class DateTransformer(BaseEstimator, TransformerMixin):
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X['DAYS'] = (pd.to_datetime('2019-11-24') - pd.to_datetime(X['UPDATED'])).dt.days
        X.drop(['UPDATED'], axis=1, inplace=True)
        return X

In [15]:
dtr = DateTransformer()
df = dtr.fit_transform(df)

In [16]:
# Works

class ToBinaryTransformer(BaseEstimator, TransformerMixin):
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X['TERMS_SALE'] = X['TERMS_SALE'].map(lambda x: 'nan' if pd.isna(x) else x)
        X['TERMS_SALE'] = X['TERMS_SALE'].map(lambda x: 1 if x.strip() == 'чистая продажа' else 0)
        
        cities = ['г. Минск', 'г. Брест',
                  'г. Гродно' ,'г. Витебск',
                  'г. Могилев', 'г. Гомель'
                  ]
        X['CITY'] = X['CITY'].map(lambda x: 1 if x in cities else 0)
        return X

In [17]:
to_bin = ToBinaryTransformer()
df = to_bin.fit_transform(df)

In [18]:
class CategoricalTransform(BaseEstimator, TransformerMixin):
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        ###########################
#         X.loc[X.VIDEO == '3D тур по недвижимости:', 'VIDEO'] = 1
#         X.loc[X.VIDEO == 'Видеоролик об объекте:', 'VIDEO'] = 2
#         X.loc[X.VIDEO != 1 and X.VIDEO != 2, 'VIDEO'] = 0

        X.loc[X.AGENCY.notna(), 'AGENCY'] = 1
        X.loc[X.AGENCY.isna(), 'AGENCY'] = 0

        X.loc[X.PROPERTY != 'частная', 'PROPERTY'] = 0
        X.loc[X.PROPERTY == 'частная', 'PROPERTY'] = 1

        X.loc[X.REPAIR == 'евроремонт', 'REPAIR'] = 3
        X.loc[X.REPAIR == 'отличный ремонт', 'REPAIR'] = 3
        X.loc[X.REPAIR == 'хороший ремонт', 'REPAIR'] = 2
        X.loc[X.REPAIR == 'нормальный ремонт', 'REPAIR'] = 1
        X.loc[X.REPAIR == 'удовлетворительный ремонт', 'REPAIR'] = 1
        X.loc[X.REPAIR == 'строительная отделка', 'REPAIR'] = 0
        X.loc[X.REPAIR == 'без отделки', 'REPAIR'] = 0
        X.loc[X.REPAIR == 'аварийное состояние', 'REPAIR'] = 0
        X.loc[X.REPAIR == 'плохое состояние', 'REPAIR'] = 0
        
        return X

In [19]:
catr = CategoricalTransform()
df = catr.fit_transform(df)

In [20]:
df.drop(['NAME', 'ADDRESS', 'PROPERTY', 'VIDEO'], axis=1, inplace=True)

In [21]:
# AREA - OHE
# DISTRICT - 
# UNDERGROUND - до 500м, больше 500м, нет
# ROOMS - parse
# HEIGHT - most frequent
# TOILET - most frequent for nan
# BALCONY - div
# POL - most frequent for nan
# 

In [22]:
df

,AREA,COORD_X,COORD_Y,CITY,DISTRICT,UNDERGROUND,ROOMS,HEIGHT,BUILD_YEAR,TOILET,BALCONY,POL,REPAIR,PRICE,TERMS_SALE,SHOP,PHARMACY,AGENCY,FULL,VIEWS_TODAY,VIEWS_LAST_WEEK,FLOOR_NOW,FLOOR_MAX,SQUARE_SUM,SQUARE_LIVE,SQUAER_KITCHEN,DAYS
0,Гродненская область,30.247803,53.927870,1,NaN,NaN,1/2 доли в 3-комнатной квартире,2.52,1980.0,раздельный,балкон и лоджия,деревянные,0,21526.0,1,200.0,500.0,0,Ленинский район: Пересёлка-4,3.0,81.0,6.0,9.0,57.0,20.0,7.0,60.0
1,Гродненская область,23.816521,53.697233,1,NaN,NaN,1/2 доли в 3-комнатной квартире,2.52,1980.0,раздельный,балкон и лоджия,деревянные,0,21526.0,1,200.0,500.0,0,Ленинский район: Пересёлка-4,4.0,82.0,6.0,9.0,57.0,20.0,7.0,60.0
2,Брестская область,30.247803,53.927870,0,NaN,NaN,47/100 доли в 2-комнатной квартире,2.50,1979.0,раздельный,лоджия,NaN,0,22551.0,1,200.0,200.0,1,NaN,6.0,69.0,5.0,5.0,49.1,14.0,7.2,93.0
3,Брестская область,24.353911,52.208929,0,NaN,NaN,47/100 доли в 2-комнатной квартире,2.50,1979.0,раздельный,лоджия,NaN,0,22551.0,1,200.0,200.0,1,NaN,8.0,71.0,5.0,5.0,49.1,14.0,7.2,93.0
4,Минская область,30.247803,53.927870,0,NaN,NaN,1/2 доли в 3-комнатной квартире,NaN,1949.0,раздельный,лоджия,деревянные,1,22551.0,1,350.0,700.0,1,NaN,12.0,114.0,2.0,4.0,71.4,48.1,9.3,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12781,Минская область,27.698900,53.946600,1,Первомайский район,800.0,8 / 8,3.00,2005,2 сан.узла,2 лоджии застекленные,NaN,0,399770.0,1,50.0,200.0,1,Первомайский район: Уручье,36.0,67.0,5.0,6.0,175.0,108.0,13.1,2.0
12782,Минская область,27.391663,53.915300,0,NaN,NaN,8 / 8,2.64,2012,2 сан.узла,лоджия,деревянные,3,861042.0,1,NaN,NaN,1,NaN,3.0,33.0,2.0,NaN,254.7,151.8,61.3,2.0
12783,Минская область,27.677200,53.946800,1,Первомайский район,700.0,8 / 7,NaN,2000,3 сан.узла,3 балкона,паркет,3,1170000.0,1,100.0,200.0,1,Первомайский район: Уручье,12.0,95.0,7.0,9.0,304.3,211.3,19.8,2.0
12784,Минская область,27.678500,53.883600,1,Заводской район,NaN,9 / 9,3.20,2010,3 сан.узла,нет,ламинированные,1,389519.0,1,300.0,200.0,1,Заводской район: Ангарская,15.0,61.0,3.0,NaN,307.4,191.3,NaN,59.0


In [23]:
df.fillna('nan')['HEIGHT'].value_counts()

nan       5500
2.5       1475
2.7       1369
2.55       391
2.6        343
          ... 
3373.0       1
3.65         1
3.12         1
3557.0       1
1.0          1
Name: HEIGHT, Length: 148, dtype: int64

In [24]:
df.loc[5, 'CITY']

0

In [25]:
df['CITY'].value_counts()

1    9076
0    3211
Name: CITY, dtype: int64

In [26]:
###########################
# df['PROPERTY'].unique()

In [27]:
df['TERMS_SALE'].unique()

array([1, 0], dtype=int64)

In [28]:
df['TERMS_SALE'].unique()

array([1, 0], dtype=int64)

In [29]:
df['TERMS_SALE'].value_counts()

1    10893
0     1394
Name: TERMS_SALE, dtype: int64

In [30]:
def prepare_coords(df):
    values = df['FLOOR'].values
    coords = [re.findall(r'\d+', i) if not pd.isna(i) else np.array(['0', '0']) for i in values]
    
    coords = [[float(j) for j in i] for i in coords]
    print(coords)
    df['COORD_X'] = coords[:][0]
    df['COORD_Y'] = coords[:, 1]
    df.drop(['coordinates'], axis=1, inplace=True)

In [31]:
###########################
#prepare_coords(df)

In [32]:
###########################
#df['ADDRESS'].value_counts()

In [33]:
###########################
df.fillna('nan')['TERMS_SALE'].value_counts()
# to binary

1    10893
0     1394
Name: TERMS_SALE, dtype: int64

In [34]:
###########################
df['BALCONY'] = df['BALCONY'].map(lambda x: 0 if pd.isna(x) or x == 'нет' else 1)

In [35]:
###########################
def height_prep(height):
    if not (0 < height < 5) and not pd.isna(height):
        if height > 1000:
            return height / 1000
        elif height < 0:
            return np.abs(height)
    elif pd.isna(height):
        return height    
    return float(height)
    
df['HEIGHT'] = df['HEIGHT'].map(height_prep)

In [36]:
###########################
df['TOILET'].value_counts()

 раздельный      7136
 совмещенный     3342
 2 сан.узла       786
 3 сан.узла        54
 4 сан.узла         5
Name: TOILET, dtype: int64

In [37]:
###########################
def ter_sale(df):
    df[df['TERMS_SALE'] == 'чистая продажа']['TERMS_SALE'] = 0
    df[df['TERMS_SALE'] != 'чистая продажа']['TERMS_SALE'] = 1
            
    return df

In [38]:
df = ter_sale(df)
df

,AREA,COORD_X,COORD_Y,CITY,DISTRICT,UNDERGROUND,ROOMS,HEIGHT,BUILD_YEAR,TOILET,BALCONY,POL,REPAIR,PRICE,TERMS_SALE,SHOP,PHARMACY,AGENCY,FULL,VIEWS_TODAY,VIEWS_LAST_WEEK,FLOOR_NOW,FLOOR_MAX,SQUARE_SUM,SQUARE_LIVE,SQUAER_KITCHEN,DAYS
0,Гродненская область,30.247803,53.927870,1,NaN,NaN,1/2 доли в 3-комнатной квартире,2.52,1980.0,раздельный,1,деревянные,0,21526.0,1,200.0,500.0,0,Ленинский район: Пересёлка-4,3.0,81.0,6.0,9.0,57.0,20.0,7.0,60.0
1,Гродненская область,23.816521,53.697233,1,NaN,NaN,1/2 доли в 3-комнатной квартире,2.52,1980.0,раздельный,1,деревянные,0,21526.0,1,200.0,500.0,0,Ленинский район: Пересёлка-4,4.0,82.0,6.0,9.0,57.0,20.0,7.0,60.0
2,Брестская область,30.247803,53.927870,0,NaN,NaN,47/100 доли в 2-комнатной квартире,2.50,1979.0,раздельный,1,NaN,0,22551.0,1,200.0,200.0,1,NaN,6.0,69.0,5.0,5.0,49.1,14.0,7.2,93.0
3,Брестская область,24.353911,52.208929,0,NaN,NaN,47/100 доли в 2-комнатной квартире,2.50,1979.0,раздельный,1,NaN,0,22551.0,1,200.0,200.0,1,NaN,8.0,71.0,5.0,5.0,49.1,14.0,7.2,93.0
4,Минская область,30.247803,53.927870,0,NaN,NaN,1/2 доли в 3-комнатной квартире,NaN,1949.0,раздельный,1,деревянные,1,22551.0,1,350.0,700.0,1,NaN,12.0,114.0,2.0,4.0,71.4,48.1,9.3,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12781,Минская область,27.698900,53.946600,1,Первомайский район,800.0,8 / 8,3.00,2005,2 сан.узла,1,NaN,0,399770.0,1,50.0,200.0,1,Первомайский район: Уручье,36.0,67.0,5.0,6.0,175.0,108.0,13.1,2.0
12782,Минская область,27.391663,53.915300,0,NaN,NaN,8 / 8,2.64,2012,2 сан.узла,1,деревянные,3,861042.0,1,NaN,NaN,1,NaN,3.0,33.0,2.0,NaN,254.7,151.8,61.3,2.0
12783,Минская область,27.677200,53.946800,1,Первомайский район,700.0,8 / 7,NaN,2000,3 сан.узла,1,паркет,3,1170000.0,1,100.0,200.0,1,Первомайский район: Уручье,12.0,95.0,7.0,9.0,304.3,211.3,19.8,2.0
12784,Минская область,27.678500,53.883600,1,Заводской район,NaN,9 / 9,3.20,2010,3 сан.узла,1,ламинированные,1,389519.0,1,300.0,200.0,1,Заводской район: Ангарская,15.0,61.0,3.0,NaN,307.4,191.3,NaN,59.0


In [39]:
###########################
#df.loc[df.video == '3D тур по недвижимости:', 'video'] = 1
#df.loc[df.video == 'Видеоролик об объекте:', 'video'] = 2
#df.loc[df.video.isna(), 'video'] = 0

df.loc[df.AGENCY.notna(), 'AGENCY'] = 1
df.loc[df.AGENCY.isna(), 'AGENCY'] = 0

#df.loc[df.PROPERTY != 'частная', 'PROPERTY'] = 0
#df.loc[df.PROPERTY == 'частная', 'PROPERTY'] = 1

df.loc[df.REPAIR == 'евроремонт', 'REPAIR'] = 3
df.loc[df.REPAIR == 'отличный ремонт', 'REPAIR'] = 3
df.loc[df.REPAIR == 'хороший ремонт', 'REPAIR'] = 2
df.loc[df.REPAIR == 'нормальный ремонт', 'REPAIR'] = 1
df.loc[df.REPAIR == 'удовлетворительный ремонт', 'REPAIR'] = 1
df.loc[df.REPAIR == 'строительная отделка', 'REPAIR'] = 0
df.loc[df.REPAIR == 'без отделки', 'REPAIR'] = 0
df.loc[df.REPAIR == 'аварийное состояние', 'REPAIR'] = 0
df.loc[df.REPAIR == 'плохое состояние', 'REPAIR'] = 0

In [48]:
df['FULL'] = df['FULL'].str.split(': ').str[0]
df['FULL']

0            Ленинский район
1            Ленинский район
2                        NaN
3                        NaN
4                        NaN
                ...         
12781     Первомайский район
12782                    NaN
12783     Первомайский район
12784        Заводской район
12785      Центральный район
Name: FULL, Length: 12287, dtype: object

In [45]:
df['ROOMS']

0           1/2 доли в 3-комнатной квартире
1           1/2 доли в 3-комнатной квартире
2        47/100 доли в 2-комнатной квартире
3        47/100 доли в 2-комнатной квартире
4           1/2 доли в 3-комнатной квартире
                        ...                
12781                                 8 / 8
12782                                 8 / 8
12783                                 8 / 7
12784                                 9 / 9
12785                                 9 / 8
Name: ROOMS, Length: 12287, dtype: object

In [47]:
def rooms_prep(room):
    if '-комнатной' in room:
        print(room)
        room = int(room.split(' ')[3].split('-')[0])
    else:
        return int(room.split('/')[1])
print(df['ROOMS'].map(rooms_prep))

1/2 доли в 3-комнатной квартире
1/2 доли в 3-комнатной квартире
47/100 доли в 2-комнатной квартире
47/100 доли в 2-комнатной квартире
1/2 доли в 3-комнатной квартире
1/2 доли в 3-комнатной квартире
комната в 2-комнатной квартире


ValueError: invalid literal for int() with base 10: 'квартире'